In [4]:
# =====================================================================
#  Match-type legend  (stored in the “match_type” column)
#  To be noticed, below 5 classes classification comes from the early stage of the research
#  at that time, our hypothesis is the system will converge to the nearest zero which is actually
#  wrong, but the absence of class (3) and (5) still make this emperical test sufficient
#  to verify C1, which is if f(δ) has zero iff the trajectory reaches a steady state.
# ---------------------------------------------------------------------
#  (1)  f(δ) has ≥1 zero  AND  the trajectory reaches a steady state
#       that matches (within MATCH_TOLERANCE) the nearest zero in the (not the zero
#       predicted by C2, but the nearest zero)
#
#  (2)  f(δ) has ≥1 zero  AND  the trajectory reaches a steady state,
#       but that value does NOT match the nearest zero.
#
#  (3)  f(δ) has ≥1 zero  BUT  no numerical steady state is detected.
#       (Would violate C1.)
#
#  (4)  f(δ) has no zeros  AND  no steady state is detected.  ← expected
#
#  (5)  f(δ) has no zeros  BUT  the solver *does* flag a steady state.
#       (Would violate C1.)
#
#  In all our runs, only (1), (2) and (4) occur, confirming empirical C1.
# =====================================================================

import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from math import sin, cos, pi
import itertools
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

# === Global configuration ===
T_MAX          = 100.0          # seconds
DT             = 0.001          # solver output step
TOLERANCE      = 5e-4           # plateau ± band (rad)
MATCH_TOLERANCE= 1e-2           # zero-match tolerance (rad)
WINDOW_SEC     = 10.0           # plateau window length
WINDOW_SIZE    = int(WINDOW_SEC / DT)
TIME_EVAL      = np.arange(0.0, T_MAX + DT, DT)

# === Parameter grid (b is fixed at 1.0) ===
delta0_vals = np.linspace(-pi,  pi, 10)
p1_vals     = np.linspace(0.0, 2.0, 10)
p2_vals     = np.linspace(0.0, 2.0, 10)
a_vals      = np.linspace(1.0, 5.0, 10)
PARAM_GRID  = list(itertools.product(delta0_vals, p1_vals, p2_vals, a_vals))

# === ODE system ===
def single_cell_ode(t, y, p1, p2, a):
    Theta, Psi = y          # ← FIXED: unpack state vector
    b = 1.0                 # minor semi-axis is held constant
    sin_term = sin(2 * (Psi - Theta))
    cos_term = cos(Psi - Theta)
    sin_diff = sin(Psi - Theta)
    denom = 2.0 * ((a * sin_diff) ** 2 + (b * cos_term) ** 2) ** 1.5
    num   = (a * b) * (a**2 - b**2) * sin_term
    dTheta_dt = -1.0 - p1 * num / denom
    dPsi_dt   = -p2 * sin_term
    return [dTheta_dt, dPsi_dt]

# === f(δ) ≔ dΨ/dt − dΘ/dt ===
def f_delta(delta, p1, p2, a):
    b = 1.0
    sin2d = np.sin(2 * delta)
    sin_d = np.sin(delta)
    cos_d = np.cos(delta)
    denom = 2.0 * ((a * sin_d) ** 2 + (b * cos_d) ** 2) ** 1.5
    dTheta_dt = -1.0 - p1 * ((a * b) * (a**2 - b**2) * sin2d) / denom
    dPsi_dt   = -p2 * sin2d
    return dPsi_dt - dTheta_dt

# === Simulation worker ===
def run_simulation(params):
    delta0, p1, p2, a = params
    Theta0 = pi / 3
    Psi0   = Theta0 + delta0
    y0     = [Theta0, Psi0]

    try:
        sol = solve_ivp(
            fun=lambda t, y: single_cell_ode(t, y, p1, p2, a),
            t_span=(0.0, T_MAX),
            y0=y0,
            t_eval=TIME_EVAL,
            method="RK45",
            rtol=1e-6,
            atol=1e-9,
        )
        if not sol.success:
            return None

        t      = sol.t
        Theta  = sol.y[0]
        Psi    = sol.y[1]
        delta  = Psi - Theta

        # --- detect steady state (10-s plateau) ---
        steady_value, steady_start_time = None, None
        for i in range(len(delta) - WINDOW_SIZE):
            window = delta[i : i + WINDOW_SIZE]
            if np.max(np.abs(window - window[0])) < TOLERANCE:
                steady_value      = window[0]
                steady_start_time = t[i]
                break

        # --- zeros of f(δ) in [δ₀ − π, δ₀ + π] ---
        delta_scan = np.linspace(delta0 - pi, delta0 + pi, 2000)
        f_vals     = f_delta(delta_scan, p1, p2, a)
        zero_cross = [
            0.5 * (delta_scan[i] + delta_scan[i + 1])
            for i in range(len(f_vals) - 1)
            if f_vals[i] * f_vals[i + 1] < 0
        ]

        has_zero               = bool(zero_cross)
        nearest_zero           = None
        dist_to_nearest_zero   = None
        match_type             = None

        if has_zero:
            zero_arr = np.asarray(zero_cross)
            nearest_zero = zero_arr[np.argmin(np.abs(zero_arr - delta0))]
            dist_to_nearest_zero = abs(delta0 - nearest_zero)
            if steady_value is not None:
                if abs(steady_value - nearest_zero) < MATCH_TOLERANCE:
                    match_type = "(1)"
                else:
                    match_type = "(2)"
            else:
                match_type = "(3)"
        else:
            match_type = "(4)" if steady_value is None else "(5)"

        return dict(
            delta0=delta0, p1=p1, p2=p2, a=a,
            has_steady=steady_value is not None,
            steady_value=steady_value,
            steady_start_time=steady_start_time,
            has_zero=has_zero,
            nearest_zero=nearest_zero,
            dist_to_nearest_zero=dist_to_nearest_zero,
            match_type=match_type,
        )

    except Exception:
        return None  # skip on any unexpected error

# === Main driver with multiprocessing ===
def main():
    with Pool(cpu_count()) as pool:
        results = []
        for res in tqdm(
            pool.imap_unordered(run_simulation, PARAM_GRID),
            total=len(PARAM_GRID),
            desc="Simulating",
        ):
            if res is not None:
                results.append(res)

    df = pd.DataFrame(results)
    df.to_excel("ode_steady_analysis.xlsx", index=False)

    # --- summary statistics ---
    summary = df["match_type"].value_counts().sort_index()
    total   = len(df)
    satisfied = summary.get("(1)", 0) + summary.get("(4)", 0)
    proportion = satisfied / total if total else 0

    print(f"\nTotal simulations: {total}")
    print(f"Satisfying C1 ((1)+(4)) : {satisfied}  ({proportion:.2%})")
    print("\nBreakdown by match type:")
    print(summary.to_string())

if __name__ == "__main__":
    main()


Simulating: 100%|██████████| 10000/10000 [04:35<00:00, 36.36it/s]



Total simulations: 10000
Satisfying C1 ((1)+(4)) : 6059  (60.59%)

Breakdown by match type:
match_type
(1)    3619
(2)    3941
(4)    2440
